# Will find correlation between ESG (and only E) scores and the return.

## Then will redefine our ESG metric, namely ESGR and maximize this variable

In [15]:
import json
import numpy as np
from scipy.stats import pearsonr
import qiskit

# Load the JSON data from a file
file_path = 'Datasets/big_dataset.json'  # Replace with your actual file path

with open(file_path, 'r') as file:
    data = json.load(file)["data"]  # Access the "data" field in the JSON

# Function to calculate the correlation between returns and ESG score for a company
def harvest_data(company_name, company_data):
    # Check if 'History' exists in the company data
    if "History" not in company_data:
        return None  # Skip if no historical data
    
    # Extract the historical closing prices
    history = company_data["History"]
    dates = sorted(history.keys())  # Sort dates chronologically
    closing_prices = [history[date]["Close"] for date in dates]
    opening_prices = [history[date]["Open"] for date in dates]

    # Calculate daily returns
    if len(closing_prices) < 2:
        return None, None, None  # Not enough data points to calculate returns
    
    daily_returns = [(closing_prices[i] - closing_prices[i - 1]) / closing_prices[i - 1] for i in range(1, len(closing_prices))]

    # Calculate annual return
    I = opening_prices[0]
    F = closing_prices[-1]
    n = 1 # one year of data
    r = (F/I)**(1/n) - 1
    
    # Check if 'Sustainability' and 'totalEsg' exist
    if "Sustainability" not in company_data or "esgScores" not in company_data["Sustainability"] or "totalEsg" not in company_data["Sustainability"]["esgScores"]:
        average_daily_return = np.mean(returns)
        return average_daily_return, None, None  # Skip if ESG data is missing

    # Extract the total ESG score
    # total_esg = company_data["Sustainability"]["esgScores"]["totalEsg"]
    total_esg = company_data["Sustainability"]["esgScores"]["peerEnvironmentPerformance"]["avg"]
    # env_score = company_data["Sustainability"]["esgScores"]["environmentScore"]
    env_score = company_data["Sustainability"]["esgScores"]["peerEsgScorePerformance"]["avg"]

    average_daily_return = np.mean(daily_returns)

    # Calculate standard deviation of daily returns
    std_dev_daily = np.std(daily_returns, ddof=1)  # ddof=1 for sample standard deviation
    
    # Annualize the standard deviation
    std_dev_annual = std_dev_daily * np.sqrt(252)
    
    return average_daily_return, std_dev_daily**2, r, std_dev_annual**2, total_esg, env_score

# Initialize a data structure to hold company metrics
company_metrics = {}

# Iterate over all companies and calculate correlations
for company_name, company_data in data.items():
    avg_daily_return, std_daily, ann_return, std_annual, esg_score, env_score = harvest_data(company_name, company_data)
    # print(f"Company: {company_name}\t avg_daily_return: {avg_daily_return:.4f}\t ann_return: {ann_return:.4f}\t esg_score: {esg_score}\t env_score: {env_score}")
    company_metrics[company_name] = {
            'avg_daily_return': avg_daily_return,  # Average return
            'std_daily': std_daily,
            'ann_return': ann_return, # Annual return
            'std_annual': std_annual,
            'esg_score': esg_score,  # Total ESG score
            'env_score': env_score   # Environmental score
        }

# Prepare lists for correlation calculation
avg_returns = []
std_dailys = []
ann_returns = []
std_annuals = []
esg_scores = []
env_scores = []

# Extracting the values for correlation
for company_name, metrics in company_metrics.items():
    avg_returns.append(metrics['avg_daily_return'])
    std_dailys.append(metrics['std_daily'])
    ann_returns.append(metrics['ann_return'])
    std_annuals.append(metrics['std_annual'])
    esg_scores.append(metrics['esg_score'])
    env_scores.append(metrics['env_score'])

# Calculate correlations
correlation_avg_return_esg, _ = pearsonr(avg_returns, esg_scores)
correlation_avg_return_env, _ = pearsonr(avg_returns, env_scores)
correlation_ann_return_esg, _ = pearsonr(ann_returns, esg_scores)
correlation_ann_return_env, _ = pearsonr(ann_returns, env_scores)
correlation_std_daily_esg, _ = pearsonr(std_dailys, esg_scores)
correlation_std_daily_env, _ = pearsonr(std_dailys, env_scores)
correlation_std_annual_esg, _ = pearsonr(std_annuals, esg_scores)
correlation_std_annual_env, _ = pearsonr(std_annuals, env_scores)

print(f"Mean E score: {sum(env_scores)/len(env_scores)}")


# # Print the results
# print(f"Correlation between avg_daily_return and esg_score: {correlation_avg_return_esg:.8f}")
# print(f"Correlation between avg_daily_return and env_score: {correlation_avg_return_env:.8f}")
# print(f"Correlation between ann_return and esg_score: {correlation_ann_return_esg:.8f}")
# print(f"Correlation between ann_return and env_score: {correlation_ann_return_env:.8f}")

# print(f"Correlation between std_daily and esg_score: {correlation_std_daily_esg:.8f}")
# print(f"Correlation between std_daily and env_score: {correlation_std_daily_env:.8f}")
# print(f"Correlation between std_annual and esg_score: {correlation_std_annual_esg:.8f}")
# print(f"Correlation between std_annual and env_score: {correlation_std_annual_env:.8f}")


Mean E score: 23.344312156182873


In [13]:
import json
from stock import Stock
import pandas as pd
import numpy as np



file_name = "Datasets/big_dataset.json"
with open(file_name, 'r') as f:
    stock_data = json.load(f)['data']
    
stocks = [Stock(stock) for stock in stock_data.values()]
# ann_returns = []
# for stock in stocks:
#     # print(stock)
#     ann_returns.append(stock.get_annualized_returns())
#     print(stock.Name,"\t", stock.get_ESR_score(),"\t", stock.get_annualized_returns())
#     # print(stock.get_annualized_returns())
#         # break
# print(np.average(ann_returns))

def ESG_scores(stocks):
    ESG = []
    for stock in stocks:
        ESG.append(stock.get_ESG_score())
    return ESG

def mean_returns(stocks):
    M =[]
    for stock in stocks:
        M.append(stock.annualized_return())
    return M

def create_cov_matrix(stocks):
    # Create a DataFrame with the closing prices of the stocks
    df = pd.concat([stock.get_price_history()['Close'] for stock in stocks], axis=1)
    df.columns = [stock.Ticker for stock in stocks]
    
    # Calculate the percentage change
    returns = df.pct_change()
    
    # Calculate the covariance matrix
    cov_matrix = returns.cov()
    
    return cov_matrix

def adjust_covariance_matrix( stocks, beta=0.1, esg_min=1):
    """
    Adjust the covariance matrix based on ESG scores, handling ESG scores of zero.
    
    Parameters:
    cov_matrix (numpy.ndarray): The original covariance matrix (N x N).
    stocks (list): A list of stock objects, where each stock has an `esg_score` attribute.
    beta (float): The scaling factor for ESG risk adjustment. Default is 0.1.
    esg_min (float): Minimum threshold for ESG scores to avoid division by zero. Default is 1.
    
    Returns:
    numpy.ndarray: The adjusted covariance matrix.
    """
    cov_matrix = create_cov_matrix(stocks)
    # Get the number of stocks
    num_stocks = len(stocks)
    
    # Calculate the ESG-based risk adjustment factor for each stock
    esg_risk_factors = np.array([1 / max(stock.get_ESR_score()[0], esg_min) for stock in stocks])
        
    # Calculate the adjusted standard deviation for each stock
    std_devs = np.sqrt(np.diag(cov_matrix))
    adjusted_std_devs = std_devs * (1 + beta * esg_risk_factors)
    
    # Initialize the adjusted covariance matrix
    adjusted_cov_matrix = pd.DataFrame(np.zeros_like(cov_matrix), index=cov_matrix.index, columns=cov_matrix.columns)
    
    # Update the adjusted covariance matrix using .iloc for integer-based indexing
    for i in range(num_stocks):
        for j in range(num_stocks):
            # Adjust covariance between stocks i and j
            adjusted_cov_matrix.iloc[i, j] = cov_matrix.iloc[i, j] * (adjusted_std_devs[i] / std_devs[i]) * (adjusted_std_devs[j] / std_devs[j])
    
    return adjusted_cov_matrix

In [4]:
create_cov_matrix(stocks)

/tmp/ipykernel_638/2674177345.py:40: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = df.pct_change()


,EXPN.L,SHEL.L,BA.L,CNA.L,TSCO.L,PRU.L,PSN.L,SDR.L,AHT.L,SSE.L,BATS.L,RTO.L,SMIN.L
EXPN.L,0.000229,0.000019,0.000018,1.273091e-05,0.000010,0.000094,0.000120,0.000102,0.000112,0.000031,1.567133e-05,0.000113,0.000060
SHEL.L,0.000019,0.000134,0.000039,3.149280e-05,0.000009,0.000039,0.000014,0.000036,0.000053,0.000013,3.078728e-05,0.000029,0.000031
BA.L,0.000018,0.000039,0.000156,4.111493e-05,0.000006,0.000019,0.000010,0.000027,0.000016,0.000011,1.643121e-05,0.000043,0.000035
CNA.L,0.000013,0.000031,0.000041,2.715830e-04,0.000018,0.000050,0.000036,0.000046,0.000015,0.000075,4.243450e-07,0.000062,0.000026
TSCO.L,0.000010,0.000009,0.000006,1.847850e-05,0.000117,0.000024,0.000032,0.000038,0.000004,0.000023,2.954806e-05,0.000030,0.000019
PRU.L,0.000094,0.000039,0.000019,5.003562e-05,0.000024,0.000357,0.000154,0.000161,0.000154,0.000049,3.367547e-05,0.000128,0.000079
PSN.L,0.000120,0.000014,0.000010,3.552879e-05,0.000032,0.000154,0.000367,0.000163,0.000166,0.000071,1.539089e-05,0.000123,0.000070
SDR.L,0.000102,0.000036,0.000027,4.638855e-05,0.000038,0.000161,0.000163,0.000246,0.000136,0.000056,2.913124e-05,0.000118,0.000080
AHT.L,0.000112,0.000053,0.000016,1.450860e-05,0.000004,0.000154,0.000166,0.000136,0.000408,0.000026,1.660657e-05,0.000120,0.000090
SSE.L,0.000031,0.000013,0.000011,7.548711e-05,0.000023,0.000049,0.000071,0.000056,0.000026,0.000141,2.281606e-05,0.000031,0.000026


In [14]:
adjust_covariance_matrix(stocks)

/tmp/ipykernel_638/4076486513.py:40: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = df.pct_change()


,AVGO,PEP,INTC,AAPL,ACN,ADP,AMAT,AMZN,BA,BAC,...,SO,TDG,TJX,TMUS,TSLA,TT,UBS,UNP,VZ,WMT
AVGO,0.000819,-4.474615e-05,0.000347,1.540239e-04,0.000072,0.000041,0.000529,0.000226,1.387370e-04,3.900302e-05,...,-0.000072,0.000176,0.000084,0.000030,0.000411,1.752769e-04,0.000110,3.722264e-05,-3.542215e-05,0.000025
PEP,-0.000045,1.137311e-04,-0.000018,-3.871179e-07,0.000018,0.000031,-0.000069,-0.000018,-6.317496e-06,2.318806e-05,...,0.000046,-0.000010,0.000020,0.000024,0.000022,-3.838972e-06,-0.000007,1.553354e-05,3.366897e-05,0.000032
INTC,0.000347,-1.804055e-05,0.001012,1.102329e-04,0.000113,0.000055,0.000396,0.000239,1.958193e-04,1.324383e-04,...,0.000008,0.000130,0.000059,0.000008,0.000326,1.450412e-04,0.000129,8.518189e-05,9.107628e-06,0.000030
AAPL,0.000154,-3.871179e-07,0.000110,2.303804e-04,0.000042,0.000018,0.000143,0.000103,6.411382e-05,2.260459e-05,...,0.000002,0.000055,0.000039,0.000003,0.000210,5.355028e-05,0.000051,2.129244e-05,-9.726396e-06,0.000005
ACN,0.000072,1.777156e-05,0.000113,4.224016e-05,0.000253,0.000041,0.000079,0.000074,4.861918e-05,3.612177e-05,...,0.000023,0.000036,0.000039,0.000019,0.000036,3.862863e-05,0.000055,4.801661e-05,1.470284e-05,0.000011
ADP,0.000041,3.115571e-05,0.000055,1.791231e-05,0.000041,0.000132,0.000031,0.000020,4.507330e-05,4.475808e-05,...,0.000025,0.000047,0.000035,0.000024,0.000044,1.237202e-05,0.000019,3.759575e-05,2.447738e-05,0.000008
AMAT,0.000529,-6.889229e-05,0.000396,1.428312e-04,0.000079,0.000031,0.000649,0.000205,1.314027e-04,8.012585e-05,...,-0.000070,0.000150,0.000063,0.000010,0.000396,1.845299e-04,0.000140,3.984167e-05,-5.340850e-05,0.000003
AMZN,0.000226,-1.816573e-05,0.000239,1.027297e-04,0.000074,0.000020,0.000205,0.000312,1.021329e-04,3.257915e-05,...,-0.000012,0.000085,0.000056,0.000010,0.000189,1.100642e-04,0.000062,2.734192e-05,-3.427898e-06,0.000039
BA,0.000139,-6.317496e-06,0.000196,6.411382e-05,0.000049,0.000045,0.000131,0.000102,4.319516e-04,9.678566e-05,...,0.000004,0.000073,0.000033,0.000020,0.000205,5.027655e-05,0.000080,5.548994e-05,2.912052e-06,0.000024
BAC,0.000039,2.318806e-05,0.000132,2.260459e-05,0.000036,0.000045,0.000080,0.000033,9.678566e-05,2.465809e-04,...,0.000050,0.000052,0.000035,0.000025,0.000127,7.029303e-05,0.000124,7.137984e-05,5.879043e-05,0.000014
